## Task 1. Stateful wordcount

In this task you're receiving batches in real time through DStream. You need to create the Wordcount program with saving and updating the state after each batch. 

You have to print the TOP-10 most popular words in the input dataset and its quantity. 

There are several points for this task:

1) You have to print data only at the end. The criteria is if you have received first empty rdd, the stream is finished. At this moment you have to print the result and stop the context.

2) You may split the line with using $flatMap$ method in Dstream.

3) In this task you need to filter out short words (with length less or equal than 3). For this aim you may use $filter$ method in Dstream.

4) Remember that you should use string lowercase. You may use $map$ method in Dstream to transform words to such case.

5) You may use  $reduceByKey$ in Dstream to merge the tuples with the same key by summing the word count value.

6) In this task, you need to be able to maintain the state across the batches. You may use the $updateStateByKey()$ method, which provides access to the state variable and help you to implement "stateful" approach. You can update the current state with results of every batch.

You may find more useful methods in the following sources:

$\cdot \quad$ Book "Learning Spark: Lightning-Fast Big Data Analysis" by Holden Karau.

$\cdot \quad \href{https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark-streaming-module}{ SparkStreaming \ documentation}$

Here you can find the starter for the main steps of the task. You can use other methods to get the solution.

In [1]:
import os
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
#!head -20 /data/wiki/en_articles_streaming/articles-part_0

**NB.** Please don't change the cell below. It is used for emulation realtime batch arriving. But figure out the code, it will help you when you'll work with real SparkStreaming applications.

In [3]:
# Preparing SparkContext
sc = SparkContext(master='local[4]')

# Preparing base RDD with the input data
DATA_PATH = "/data/wiki/en_articles_streaming"

batches = [sc.textFile(os.path.join(DATA_PATH, path)) for path in os.listdir(DATA_PATH)]

# Creating Dstream to emulate realtime data generating
BATCH_TIMEOUT = 5  # Timeout between batch generation
ssc = StreamingContext(sc, BATCH_TIMEOUT)
dstream = ssc.queueStream(rdds=batches)

In [4]:
finished = False
printed = False


def set_ending_flag(rdd):
    global finished
    if rdd.isEmpty():
        finished = True


def print_only_at_the_end(rdd):
    global printed
    
    rdd.count()
    #Transformations are executing lazy, immutable and produces new rdd during executing Directed acyclic graph flow, once action is provided 
    #So the transformation is not executed till we get some action
    #So the transformation is not executed till we get some action, so In my case, no transformation won't be executed till it pass the if case, and once it pass the if, there is an action called *.collect()*
    #Inside the transformations in the dag we still count the dstream transformations, because they are actually in the RDD
    if finished and not printed:

        ans = rdd.sortBy(lambda x : x[1], ascending=False).collect()
   
        for key in ans[:10]:
            print(key[0], "\t", key[1])
             
        printed = True

# If we have received an empty rdd, the stream is finished.
# So print the result and stop the context.
dstream.foreachRDD(set_ending_flag)

In [5]:
def aggregator(values, old):
    return (old or 0) + sum(values)

In [6]:
import re
def parse_article(line):
    try:
        article_id, text = line.rstrip().lower().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

In [7]:
# line = "12	Anarchism         Anarchism is often defined as a political philosophy which holds the state to be undesirable, unnecessary, or harmful.          The following sources cite anarchism as a political philosophy:      Slevin, Carl. \"Anarchism.\""

# print(parse_article(line))

In [8]:
# Type your code for data processing and aggregation here

dstream.flatMap(lambda line: parse_article(line))\
    .filter(lambda word : len(word)>=4)\
    .map(lambda word: (word.lower(), 1))\
    .reduceByKey(lambda a,b : a+b)\
    .updateStateByKey(aggregator)\
    .foreachRDD(print_only_at_the_end)

**NB.** Please don't change the cell below. It is used for stopping SparkStreaming context and Spark context when the stream finished.

In [9]:
ssc.checkpoint('./checkpoint')  # checkpoint for storing current state        
ssc.start()
while not printed:
    pass
ssc.stop()  # when the result printed, stop the SparkStreaming context
sc.stop()  # stop the Spark context to be able restart the code without restarting the kernel

that 	 81572
with 	 79559
from 	 58201
which 	 42198
this 	 38252
were 	 34403
also 	 31574
have 	 29871
their 	 24579
other 	 23538


Here you can see the part of an output on the sample dataset:

```
...
which 42198
this 38252
were 34403
...
```

Of course, the numbers may be different but not very much (the error about 1% will be accepted).